In [1]:
# Import Library
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

### 1.1 지하철 역 정보 set 전처리

#### 지하철역 정보 set 재인코딩 후 저장

In [2]:
"""
sta_info = pd.read_csv('지하철 역 정보.csv', encoding='utf-8')
sta_info.to_csv('지하철 역 정보.csv', index=False, encoding='utf-8-sig')
"""

"\nsta_info = pd.read_csv('지하철 역 정보.csv', encoding='utf-8')\nsta_info.to_csv('지하철 역 정보.csv', index=False, encoding='utf-8-sig')\n"

In [3]:
# Loading Data
sta_info = pd.read_csv('지하철 역 정보.csv')
sta_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 40 columns):
SBW_STATN_ID                 1014 non-null int64
SBW_STATN_NM                 1014 non-null object
SBW_STATN_NCNAM              213 non-null object
SBW_STATN_XCRD               1014 non-null int64
SBW_STATN_YCRD               1014 non-null int64
NEXT_STATN_ID1               1014 non-null int64
NEXT_STATN_NM1               973 non-null object
NEXT_STATN_ID2               1014 non-null int64
NEXT_STATN_NM2               8 non-null object
BFR_STATN_ID1                1014 non-null int64
BFR_STATN_NM1                980 non-null object
BFR_STATN_ID2                1014 non-null int64
BFR_STATN_NM2                2 non-null object
TRSF_STATN_ID1               1014 non-null int64
TRSF_ROUTE_ID1               1014 non-null int64
TRSF_STATN_ID2               1014 non-null int64
TRSF_ROUTE_ID2               1014 non-null int64
TRSF_STATN_ID3               1014 non-null int64
TRSF_ROUTE_ID3

In [4]:
# 역이름, 역주소
col = ['SBW_STATN_NM','SBW_STATN_ADDR']
sta_addr = sta_info[col]
sta_addr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 2 columns):
SBW_STATN_NM      1014 non-null object
SBW_STATN_ADDR    1014 non-null object
dtypes: object(2)
memory usage: 15.9+ KB


In [5]:
"""
# 역코드 데이터타입 변환(int64 -> int32)
sta_addr['SBW_STATN_ID'] = sta_addr['SBW_STATN_ID'].astype('int32')
sta_addr.info()
"""

"\n# 역코드 데이터타입 변환(int64 -> int32)\nsta_addr['SBW_STATN_ID'] = sta_addr['SBW_STATN_ID'].astype('int32')\nsta_addr.info()\n"

In [6]:
# 서울시 데이터만 추출
sta_seoul = sta_addr[sta_addr['SBW_STATN_ADDR'].str.startswith("서울")]
sta_seoul.head()

,SBW_STATN_NM,SBW_STATN_ADDR
27,도봉산,서울특별시 도봉구 도봉동 18-1
28,도봉,서울특별시 도봉구 도봉동 639
29,방학,서울특별시 도봉구 방학동 728
30,창동,서울특별시 도봉구 창동 135-1
31,녹천,서울특별시 도봉구 창동 57-11


In [7]:
# 역 주소 시(SI), 구(GU), 동(DONG), 번지(NUM)로 나누기
sta_seoul['SI'],sta_seoul['GU'],sta_seoul['DONG'],sta_seoul['NUM'] = sta_seoul['SBW_STATN_ADDR'].astype(str).str.split(' ',3).str
sta_seoul.head()

,SBW_STATN_NM,SBW_STATN_ADDR,SI,GU,DONG,NUM
27,도봉산,서울특별시 도봉구 도봉동 18-1,서울특별시,도봉구,도봉동,18-1
28,도봉,서울특별시 도봉구 도봉동 639,서울특별시,도봉구,도봉동,639
29,방학,서울특별시 도봉구 방학동 728,서울특별시,도봉구,방학동,728
30,창동,서울특별시 도봉구 창동 135-1,서울특별시,도봉구,창동,135-1
31,녹천,서울특별시 도봉구 창동 57-11,서울특별시,도봉구,창동,57-11


In [8]:
# 역이름에 '~역' 붙이기
sta_seoul['SBW_STATN_NM'] = sta_seoul['SBW_STATN_NM'][:-1] + '역'
sta_seoul.head()

,SBW_STATN_NM,SBW_STATN_ADDR,SI,GU,DONG,NUM
27,도봉산역,서울특별시 도봉구 도봉동 18-1,서울특별시,도봉구,도봉동,18-1
28,도봉역,서울특별시 도봉구 도봉동 639,서울특별시,도봉구,도봉동,639
29,방학역,서울특별시 도봉구 방학동 728,서울특별시,도봉구,방학동,728
30,창동역,서울특별시 도봉구 창동 135-1,서울특별시,도봉구,창동,135-1
31,녹천역,서울특별시 도봉구 창동 57-11,서울특별시,도봉구,창동,57-11


In [9]:
# 필요없는 컬럼 제거
col = ['SBW_STATN_ADDR','SI','GU','NUM']
sta_seoul = sta_seoul.drop(col, axis='columns')

# 컬럼명 변경(DONG -> 법정동명)
sta_seoul = sta_seoul.rename({'DONG':'법정동명'}, axis='columns')

sta_seoul.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 27 to 689
Data columns (total 2 columns):
SBW_STATN_NM    389 non-null object
법정동명            390 non-null object
dtypes: object(2)
memory usage: 9.1+ KB


In [10]:
sta_seoul.to_csv('지하철역정보_서울만.csv', index=False, encoding='utf-8-sig')

### 1.2 법정동/행정동 코드&명 set 전처리

In [11]:
code_info = pd.read_csv('행정동_법정동코드.csv', encoding='ansi')
code_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767 entries, 0 to 766
Data columns (total 7 columns):
행정동코드    767 non-null int64
시도명      767 non-null object
시군구명     767 non-null object
읍면동명     742 non-null object
법정동코드    767 non-null int64
동리명      767 non-null object
생성일자     767 non-null int64
dtypes: int64(3), object(4)
memory usage: 42.0+ KB


In [12]:
# 필요없는 컬럼 제거
col = ['시도명','시군구명','생성일자']
code_info = code_info.drop(col, axis='columns')

# 컬럼명 수정(읍면동명 -> 행정동명 / 동리명 -> 법정동명)
code_info = code_info.rename({'읍면동명':'행정동명', '동리명':'법정동명'}, axis='columns')

# 컬럼 데이터타입 변환(int64 -> int32)
col = ['행정동코드', '법정동코드']
code_info[col] = code_info[col].astype('int32')

In [13]:
code_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767 entries, 0 to 766
Data columns (total 4 columns):
행정동코드    767 non-null int32
행정동명     742 non-null object
법정동코드    767 non-null int32
법정동명     767 non-null object
dtypes: int32(2), object(2)
memory usage: 18.1+ KB


### 1.3 지하철역 정보 set과 코드 정보 set 병합

In [14]:
# sta_seoul + code_info
sta_code = pd.merge(sta_seoul, code_info, how='left', on='법정동명')
sta_code.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1042 entries, 0 to 1041
Data columns (total 5 columns):
SBW_STATN_NM    1038 non-null object
법정동명            1042 non-null object
행정동코드           1040 non-null float64
행정동명            1040 non-null object
법정동코드           1040 non-null float64
dtypes: float64(2), object(3)
memory usage: 48.8+ KB


In [15]:
# 데이터 타입 변환 (float64 -> int32)
col = ['법정동코드','행정동코드']
sta_code[col] = sta_code[col].fillna(0).astype('int32')

In [16]:
# '역명' NaN인 값 행 제거
sta_code = sta_code.dropna(subset=['SBW_STATN_NM'])

In [17]:
# 지하철 정보 set + 코드 정보 set 파일 저장
sta_code.to_csv('지하철역정보_행법정동코드_전처리.csv', index=False, encoding='utf-8-sig')

In [18]:
"""
# 피클데이터로 저장
import joblib

with open('지하철역정보_행법정동코드_전처리', 'wb') as f:
    joblib.dump(sta_code, f)
"""

"\n# 피클데이터로 저장\nimport joblib\n\nwith open('지하철역정보_행법정동코드_전처리', 'wb') as f:\n    joblib.dump(sta_code, f)\n"

In [19]:
"""
# 피클 데이터 불러오기
with open('지하철역정보_행법정동코드_전처리', 'rb') as f:
    sta_code = joblib.load(f)
"""

"\n# 피클 데이터 불러오기\nwith open('지하철역정보_행법정동코드_전처리', 'rb') as f:\n    sta_code = joblib.load(f)\n"